In [810]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pandas.api.types import is_string_dtype, is_object_dtype, is_categorical_dtype, is_bool_dtype
import numpy as np
import pandas as pd

import statsmodels.api as sm
from scipy.stats import spearmanr
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

from collections import OrderedDict

from sklearn.linear_model import LinearRegression, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from timeit import default_timer as timer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_boston
from pandas.api.types import is_string_dtype, is_object_dtype, is_categorical_dtype, is_bool_dtype
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model


In [63]:
df = pd.read_csv("/tmp/t.csv")
df.sort_values('Importance', ascending=False)

FileNotFoundError: [Errno 2] File b'/tmp/t.csv' does not exist: b'/tmp/t.csv'

In [149]:
R = df.copy()
R['clipped'] = np.where(R['Sigma']<0.007, 1, R['Sigma'])
R['Rank'] = R['Importance'] / R['clipped']
R.sort_values('Rank',ascending=False)

,Feature,Rank,Importance,Sigma,clipped
2,ModelID,10.013405,0.165620,0.016540,0.016540
6,ProductSize,9.710048,0.078312,0.008065,0.008065
1,YearMade,3.968520,0.304781,0.076800,0.076800
8,saledayofweek,3.431516,0.025268,0.007364,0.007364
9,salemonth,1.988941,0.015781,0.007935,0.007935
0,age,1.812289,0.194837,0.107509,0.107509
5,saledayofyear,1.736897,0.044144,0.025415,0.025415
4,auctioneerID,1.571823,0.042153,0.026818,0.026818
10,saleday,1.361876,0.010579,0.007768,0.007768
3,MachineHours,1.304369,0.051107,0.039181,0.039181


In [146]:
np.mean(df['Sigma']), np.std(df['Sigma'])

(0.022317773175551302, 0.02996326640295244)

In [191]:
h = np.array([[0,4,2,np.nan],
              [np.nan,0,3,5],
              [6,0,8,np.nan]])
h = h.T
h

array([[ 0., nan,  6.],
       [ 4.,  0.,  0.],
       [ 2.,  3.,  8.],
       [nan,  5., nan]])

In [ ]:
h = np.array([[1,0,np.nan,np.nan],
              [0,4,2,np.nan],
              [np.nan,0,3,5],
              [6,0,8,np.nan],
              [1,np.nan,np.nan,0]])
h = h.T
h

r = np.array([1,0,1,1,3]) # relative to this row

In [239]:
h[:,np.where(r==1)[0]] # columns relative to 1

array([[ 1., nan,  6.],
       [ 0.,  0.,  0.],
       [nan,  3.,  8.],
       [nan,  5., nan]])

In [240]:
s = np.nansum(h[:,np.where(r==1)[0]], axis=1)
s

array([ 7.,  0., 11.,  5.])

In [241]:
c = (~np.isnan(h[:,np.where(r==1)[0]])).astype(int)
c

array([[1, 0, 1],
       [1, 1, 1],
       [0, 1, 1],
       [0, 1, 0]])

In [242]:
c = np.sum(c, axis=1) # can ignore element at current relative pos==1
c[1] = 0
c

array([2, 0, 2, 1])

In [243]:
s/c

/Users/parrt/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([3.5, nan, 5.5, 5. ])

In [424]:
ncats=h.shape[0]
nleaves=h.shape[1]
catcodes=[0,1,2,3]
sums = []
counts = []
# result = np.zeros(shape=(ncats,))
for i in sorted(np.unique(r)):
    print("cat",i,"relative to", result[i], "for", h[:,np.where(r==i)[0]])
    s = np.nansum(h[:,np.where(r==i)[0]], axis=1)
    s[i] = 0.0
    c = (h[:,np.where(r==i)[0]]>0).astype(int)
    c = np.sum(c, axis=1) # can ignore element at current relative pos==i
    counts.append(c)
    sums.append(s)
    print("count",c)
    print("sum",s)
#     print("avg",s/c)
    result += s
print()
print("sums\n", np.array(sums).T)
print("counts\n",np.array(counts).T)
print("uniq r=",np.unique(r))

cat 0 relative to 3885.0 for [[ 0.]
 [ 4.]
 [ 2.]
 [nan]]
count [0 1 1 0]
sum [0. 4. 2. 0.]
cat 1 relative to 974.0 for [[ 1. nan  6.]
 [ 0.  0.  0.]
 [nan  3.  8.]
 [nan  5. nan]]
count [2 0 2 1]
sum [ 7.  0. 11.  5.]
cat 3 relative to 1222.0 for [[ 1.]
 [nan]
 [nan]
 [ 0.]]
count [1 0 0 0]
sum [1. 0. 0. 0.]

sums
 [[ 0.  7.  1.]
 [ 4.  0.  0.]
 [ 2. 11.  0.]
 [ 0.  5.  0.]]
counts
 [[0 2 1]
 [1 0 0]
 [1 2 0]
 [0 1 0]]
uniq r= [0 1 3]


/Users/parrt/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in greater
  # This is added back by InteractiveShellApp.init_path()


In [425]:
a = (np.array(sums)/np.array(counts)).T
#a[np.isnan(a)] = 0
a

/Users/parrt/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([[nan, 3.5, 1. ],
       [4. , nan, nan],
       [2. , 5.5, nan],
       [nan, 5. , nan]])

In [426]:
uniq_r = sorted(np.unique(r))
x = a[:,0] # init with first col
x

array([nan,  4.,  2., nan])

In [419]:
adjusted_vec = x[uniq_r[1]] + a[:,1]
#adjusted_vec[uniq_r[1]] = 0
adjusted_vec

array([7.5, nan, 9.5, 9. ])

In [420]:
x = np.where(np.isnan(x), 0, x) + np.where(np.isnan(adjusted_vec), 0, adjusted_vec)
x

array([ 7.5,  4. , 11.5,  9. ])

In [421]:
adjusted_vec = x[uniq_r[2]] + a[:,2]
#adjusted_vec[uniq_r[2]] = 0
adjusted_vec

array([10., nan, nan, nan])

In [422]:
x = np.where(np.isnan(x), 0, x) + np.where(np.isnan(adjusted_vec), 0, adjusted_vec)
x

array([17.5,  4. , 11.5,  9. ])

In [432]:
a = (np.array(sums)/np.array(counts)).T
uniq_r = sorted(np.unique(r))
x = a[:,uniq_r[0]] # init with first col
print(uniq_r[0],":",x)
for i in range(1,len(uniq_r)):
    adjusted_vec = x[uniq_r[i]] + a[:,i]
    x = np.where(np.isnan(x), 0, x) + np.where(np.isnan(adjusted_vec), 0, adjusted_vec)
    print(uniq_r[i],":",adjusted_vec,"x =",x)
x

0 : [nan  4.  2. nan]
1 : [7.5 nan 9.5 9. ] x = [ 7.5  4.  11.5  9. ]
3 : [10. nan nan nan] x = [17.5  4.  11.5  9. ]


/Users/parrt/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([17.5,  4. , 11.5,  9. ])

In [452]:
def avg_values_at_cat(leaf_histos, min_cats):
    sums = []
    counts = []
    # result = np.zeros(shape=(ncats,))
    for i in sorted(np.unique(min_cats)):
        s = np.nansum(leaf_histos[:, np.where(min_cats == i)[0]], axis=1)
#        s[i] = 0.0
        c = (leaf_histos[:, np.where(min_cats == i)[0]] > 0).astype(int)
        c = np.sum(c, axis=1)  # can ignore element at current relative pos==i
        counts.append(c)
        sums.append(s)
        # print("count", c)
        # print("sum", s)
        #     print("avg",s/c)
    print()
    print("list sums",sums)
    print("list counts", counts)
    print("sums\n", np.array(sums).T)
    print("counts\n", np.array(counts).T)
    print("uniq r=", np.unique(min_cats))
    uniq_min_cats = sorted(np.unique(min_cats))
#    a = (np.array(sums)/np.array(counts)).T
    a = [sums[i]/counts[i] for i in range(len(uniq_min_cats))]
    x = a[uniq_min_cats[0]] # init with first col
    print(uniq_min_cats[0],":",x)
    for i in range(1,len(uniq_min_cats)):
        adjusted_vec = x[uniq_min_cats[i]] + a[i]
        x = np.where(np.isnan(x), 0, x) + np.where(np.isnan(adjusted_vec), 0, adjusted_vec)
        print(uniq_min_cats[i],":",adjusted_vec,"x =",x)
    print(x)
    return x

In [453]:
leaf_histos = np.array([[1,0,np.nan,np.nan],
                      [0,4,2,np.nan],
                      [np.nan,0,3,5],
                      [6,0,8,np.nan],
                      [1,np.nan,np.nan,0]])
leaf_histos = leaf_histos.T
print(leaf_histos)

min_cats = np.array([1,0,1,1,3]) # relative to this row
avg_values_at_cat(leaf_histos, min_cats)

[[ 1.  0. nan  6.  1.]
 [ 0.  4.  0.  0. nan]
 [nan  2.  3.  8. nan]
 [nan nan  5. nan  0.]]

list sums [array([0., 4., 2., 0.]), array([ 7.,  0., 11.,  5.]), array([1., 0., 0., 0.])]
list counts [array([0, 1, 1, 0]), array([2, 0, 2, 1]), array([1, 0, 0, 0])]
sums
 [[ 0.  7.  1.]
 [ 4.  0.  0.]
 [ 2. 11.  0.]
 [ 0.  5.  0.]]
counts
 [[0 2 1]
 [1 0 0]
 [1 2 0]
 [0 1 0]]
uniq r= [0 1 3]
0 : [nan  4.  2. nan]
1 : [7.5 nan 9.5 9. ] x = [ 7.5  4.  11.5  9. ]
3 : [10. nan nan nan] x = [17.5  4.  11.5  9. ]
[17.5  4.  11.5  9. ]


/Users/parrt/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater
  
/Users/parrt/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


array([17.5,  4. , 11.5,  9. ])

In [511]:
a = np.array([3,3,3,2,3,4,4,8,1,2,1,1,1,3,2,2,2,2])
a *= 10

In [512]:
uniq, counts = np.unique(a, return_counts=True)

In [513]:
uniq

array([10, 20, 30, 40, 80])

In [514]:
counts

array([4, 6, 5, 2, 1])

In [515]:
revsort_idx = np.argsort(counts)[::-1]
revsort_idx

array([1, 2, 0, 3, 4])

In [528]:
cats_by_most_common = list(uniq[revsort_idx])
cats_by_most_common

[20, 30, 10, 40, 80]

In [531]:
leaf_cats = np.array([40,20])

In [532]:
leaf_cat_idxs = [list(cats_by_most_common).index(cat) for cat in leaf_cats]
leaf_cat_idxs

[3, 0]

In [533]:
most_common_leaf_cat = cats_by_most_common[np.min(leaf_cat_idxs)]
most_common_leaf_cat

20

In [51]:
df = pd.DataFrame()
df['x1'] = [1,2,3]
df['y']  = [100,100,100]
#df['y'] -= 150
df

,x1,y
0,1,100
1,2,100
2,3,100


In [52]:
df2 = pd.DataFrame()
df2['x1'] = [3,4,5]
df2['y']  = [200,200,200]
#df2['y'] -= 150
df2

,x1,y
0,3,200
1,4,200
2,5,200


In [53]:
df3 = df.merge(df2, on='x1', how='outer')
#df3['y'] = df3['y_x'] + df3['y_y']
df3['y'] = [-50,-50,0,50,50]
df3 = df3[['x1','y']]
print(df3)

   x1   y
0   1 -50
1   2 -50
2   3   0
3   4  50
4   5  50


In [71]:
def load_flights(n):
    """
    Download from https://www.kaggle.com/usdot/flight-delays/download and save
    flight-delays.zip; unzip to convenient data dir.  Save time by storing as
    feather.  5.8M records.
    """
    dir = "/Users/parrt/data/flight-delays"
    if os.path.exists(dir+"/flights.feather"):
        df_flights = pd.read_feather(dir + "/flights.feather")
    else:
        df_flights = pd.read_csv(dir+"/flights.csv", low_memory=False)
        df_flights.to_feather(dir+"/flights.feather")

    df_flights['dayofyear'] = pd.to_datetime(
        df_flights[['YEAR', 'MONTH', 'DAY']]).dt.dayofyear
    df_flights = df_flights[
        (df_flights['CANCELLED'] == 0) & (df_flights['DIVERTED'] == 0)]

    # times are in 830 to mean 08:30, convert to two columns, hour and min
    def cvt_time(df, colname):
        df[f'{colname}_HOUR'] = df[colname] / 100
        df[f'{colname}_HOUR'] = df[f'{colname}_HOUR'].astype(int)
        df[f'{colname}_MIN']  = df[colname] - df[f'{colname}_HOUR'] * 100
        df[f'{colname}_MIN']  = df[f'{colname}_MIN'].astype(int)

    # cvt_time(df_flights, 'SCHEDULED_DEPARTURE')
    # cvt_time(df_flights, 'SCHEDULED_ARRIVAL')
    # cvt_time(df_flights, 'DEPARTURE_TIME')

    features = [#'YEAR',  # drop year as it's a constant
                'MONTH', 'DAY', 'DAY_OF_WEEK', 'dayofyear',
                'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
                'SCHEDULED_DEPARTURE',
                # 'SCHEDULED_DEPARTURE_HOUR', 'SCHEDULED_DEPARTURE_MIN',
                'SCHEDULED_ARRIVAL',
                # 'SCHEDULED_ARRIVAL_HOUR',   'SCHEDULED_ARRIVAL_MIN',
                'DEPARTURE_TIME',
                # 'DEPARTURE_TIME_HOUR',      'DEPARTURE_TIME_MIN',
                'FLIGHT_NUMBER', 'TAIL_NUMBER',
                'AIR_TIME', 'DISTANCE',
                'TAXI_IN', 'TAXI_OUT',
                'SCHEDULED_TIME',
                'ARRIVAL_DELAY']  # target

    df_flights = df_flights[features]
    df_flights = df_flights.dropna()  # ignore missing stuff for ease and reduce size
    df_flights = df_flights.sample(n)
    df_string_to_cat(df_flights)
    df_cat_to_catcode(df_flights)

    X, y = df_flights.drop('ARRIVAL_DELAY', axis=1), df_flights['ARRIVAL_DELAY']

    return X, y, df_flights

def df_string_to_cat(df:pd.DataFrame) -> dict:
    catencoders = {}
    for colname in df.columns:
        if is_string_dtype(df[colname]) or is_object_dtype(df[colname]):
            df[colname] = df[colname].astype('category').cat.as_ordered()
            catencoders[colname] = df[colname].cat.categories
    return catencoders


def df_cat_to_catcode(df):
    for col in df.columns:
        if is_categorical_dtype(df[col]):
            df[col] = df[col].cat.codes + 1


def fix_missing_num(df, colname):
    df[colname+'_na'] = pd.isnull(df[colname]).astype(int)
    df[colname].fillna(df[colname].median(), inplace=True)


def df_split_dates(df,colname):
    df["saleyear"] = df[colname].dt.year
    df["salemonth"] = df[colname].dt.month
    df["saleday"] = df[colname].dt.day
    df["saledayofweek"] = df[colname].dt.dayofweek
    df["saledayofyear"] = df[colname].dt.dayofyear
    df[colname] = df[colname].astype(np.int64) # convert to seconds since 1970


In [114]:
X, y, df_flights = load_flights(1000)

In [115]:
catcolnames={'AIRLINE',
          'ORIGIN_AIRPORT',
          'DESTINATION_AIRPORT',
          'FLIGHT_NUMBER',
          'DAY_OF_WEEK'}
features=df_flights.columns
df = pd.DataFrame(X, columns=features)
#df

In [123]:
for cat in catcolnames:
    df[cat] = df[cat].astype('category').cat.as_ordered()
dummies = pd.get_dummies(df)
X = pd.concat([df, dummies], axis=1)
X = X.drop(catcolnames, axis=1)

In [128]:
X = X.fillna(0)  # flip missing numeric values to zeros

In [129]:
np.isnan(X.values).any()

False

In [131]:
m = LinearRegression()
m.fit(X, y)
y_pred = m.predict(X)
mean_absolute_error(y, y_pred)

6.198033730697716e-11

## Basketball

In [108]:
df = pd.DataFrame(data=[84,72,73,80,82], index=['Ctr','Guard1','Guard2','Forward1','Forward2'],
                  columns=['Height'])
df

,Height
Ctr,84
Guard1,72
Guard2,73
Forward1,80
Forward2,82


In [109]:
df['Ctr Delta'] = df['Height'] - df.loc['Ctr','Height']
df['G1 Delta'] = df['Height'] - df.loc['Guard1','Height']
df['G2 Delta'] = df['Height'] - df.loc['Guard2','Height']
df['F1 Delta'] = df['Height'] - df.loc['Forward1','Height']
df['F2 Delta'] = df['Height'] - df.loc['Forward2','Height']
df

,Height,Ctr Delta,G1 Delta,G2 Delta,F1 Delta,F2 Delta
Ctr,84,0,12,11,4,2
Guard1,72,-12,0,-1,-8,-10
Guard2,73,-11,1,0,-7,-9
Forward1,80,-4,8,7,0,-2
Forward2,82,-2,10,9,2,0


In [110]:
U = np.triu(df.iloc[:,1:])
U

array([[  0,  12,  11,   4,   2],
       [  0,   0,  -1,  -8, -10],
       [  0,   0,   0,  -7,  -9],
       [  0,   0,   0,   0,  -2],
       [  0,   0,   0,   0,   0]])

In [111]:
np.sum(np.abs(U))/(5*4/2), np.mean(np.abs(U))

(6.6, 2.64)

In [112]:
df.loc['Avg'] = np.mean(df,axis=0)
df

,Height,Ctr Delta,G1 Delta,G2 Delta,F1 Delta,F2 Delta
Ctr,84.0,0.0,12.0,11.0,4.0,2.0
Guard1,72.0,-12.0,0.0,-1.0,-8.0,-10.0
Guard2,73.0,-11.0,1.0,0.0,-7.0,-9.0
Forward1,80.0,-4.0,8.0,7.0,0.0,-2.0
Forward2,82.0,-2.0,10.0,9.0,2.0,0.0
Avg,78.2,-5.8,6.2,5.2,-1.8,-3.8


## Weighted cat impact

In [163]:
def all_pairs_abs_delta(avg_per_cat):
    """
    Impact for a categorical variable x_j is the average magnitude of change of
    PD_j from category level A to level B, for all possible pairs A,B. NaN entries
    indicate we have no data for that category and so these are ignored.

    Mathematically, we are filling an upper triangular matrix by putting the raw
    avg_per_cat as first row, then shifting that vector by k=1, 2, 3, ... to get
    the other rows of the matrix. Then, we take the nanmean of the upper triangular
    part. The lower triangular part is the negative of the upper and the diagonal is 0

    See test_avg_all_pairs_abs_delta.py for unit tests.
    """
    # first find catcodes (same as index into avg_per_cat). Find all non-NaN
    catcodes = np.where(~np.isnan(avg_per_cat))[0]
    all_pairwise_deltas = []
    for k in catcodes:
        # Shift cat k to 0 to find relative bump up/down to others;
        # Since not all cats are present, some of these are still NaN
        rel_to_k = avg_per_cat - avg_per_cat[k]
        all_pairwise_deltas.extend(rel_to_k[k+1:])
        # print(rel_to_k[k+1:])
    # print(len(avg_per_cat), len(all_pairwise_deltas))
    return all_pairwise_deltas

In [124]:
dy = np.array([0.0,12.0,11.0,4.0,2.0])
I = np.array([dy, dy - dy[1], dy - dy[2], dy - dy[3], dy - dy[4]])
I = pd.DataFrame(I, columns=['A','B','C','D','E'], index=['A','B','C','D','E'])
I

,A,B,C,D,E
A,0.0,12.0,11.0,4.0,2.0
B,-12.0,0.0,-1.0,-8.0,-10.0
C,-11.0,1.0,0.0,-7.0,-9.0
D,-4.0,8.0,7.0,0.0,-2.0
E,-2.0,10.0,9.0,2.0,0.0


In [200]:
np.sum(np.abs(I.values)) / (25-5)

6.6

In [205]:
w = [100,15,78,910,421]
np.sum(np.abs(I.values * w)) / ((25-5)*np.sum(w))

1.126706036745407

In [161]:
np.triu(I)

array([[  0.,  12.,  11.,   4.,   2.],
       [  0.,   0.,  -1.,  -8., -10.],
       [  0.,   0.,   0.,  -7.,  -9.],
       [  0.,   0.,   0.,   0.,  -2.],
       [  0.,   0.,   0.,   0.,   0.]])

In [168]:
pairs = all_pairs_abs_delta(np.array([  0.,  12.,  11.,   4.,   2.]))
pairs

[12.0, 11.0, 4.0, 2.0, -1.0, -8.0, -10.0, -7.0, -9.0, -2.0]

In [169]:
np.mean(np.abs(pairs))

6.6

In [176]:
np.sum(np.abs(np.triu(I))) / 10

6.6

In [179]:
np.sum(np.abs(I.values)) / 20

6.6

In [170]:
np.mean(np.abs(np.triu(I)))

2.64

In [174]:
np.mean(np.abs(I.values))/2

2.64

In [197]:
I_ = I.copy()
I_['avg'] = np.mean(I, axis=1)
I_ = I_.append(np.sum(I, axis=0)/(5-1), ignore_index=True)
I_['names'] = ['A','B','C','D','E','avg']
I_ = I_.set_index('names')
I_

,A,B,C,D,E,avg
names,,,,,,
A,0.00,12.00,11.0,4.00,2.00,5.8
B,-12.00,0.00,-1.0,-8.00,-10.00,-6.2
C,-11.00,1.00,0.0,-7.00,-9.00,-5.2
D,-4.00,8.00,7.0,0.00,-2.00,1.8
E,-2.00,10.00,9.0,2.00,0.00,3.8
avg,-7.25,7.75,6.5,-2.25,-4.75,NaN


## City population

In [89]:
cities1 = ['NYC','SF','LA','Boston','Peoria']
pop1   =  [10,.8,14,3,.1]
cities2 = ['NYC','Boston','Paris','London','Bruge','Lyon']
pop2   =  [10,3,7,9,.4, 1.3]

cities = set(cities1).union(set(cities2))
df = pd.DataFrame(index=cities)
df['Group1'] = pd.Series(pop1,index=cities1)
df['Group2'] = pd.Series(pop2,index=cities2)
df['Avg'] = np.nanmean(df,axis=1)
print("NOISELESS, COMMON CITIES HAVE SAME VALUES")
df

NOISELESS, COMMON CITIES HAVE SAME VALUES


,Group1,Group2,Avg
SF,0.8,NaN,0.8
LA,14.0,NaN,14.0
Peoria,0.1,NaN,0.1
Lyon,NaN,1.3,1.3
Paris,NaN,7.0,7.0
Bruge,NaN,0.4,0.4
London,NaN,9.0,9.0
NYC,10.0,10.0,10.0
Boston,3.0,3.0,3.0


### Make deltas now

In [90]:
df['Group1'] -= np.min(df['Group1'])
df['Group2'] -= np.min(df['Group2'])
df['Avg'] = np.nanmean(df,axis=1)
#df = df.drop('Avg', axis=1)
df

,Group1,Group2,Avg
SF,0.7,NaN,0.750000
LA,13.9,NaN,13.950000
Peoria,0.0,NaN,0.050000
Lyon,NaN,0.9,1.100000
Paris,NaN,6.6,6.800000
Bruge,NaN,0.0,0.200000
London,NaN,8.6,8.800000
NYC,9.9,9.6,9.833333
Boston,2.9,2.6,2.833333


### Merge by NYC

Make NYC 0 point for both

In [92]:
df['Group1'] -= df.loc['NYC','Group1']
df['Group2'] -= df.loc['NYC','Group2']
df['Avg'] = np.nanmean(df.iloc[:,0:1],axis=1)
df

/Users/parrt/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Mean of empty slice
  This is separate from the ipykernel package so we can avoid doing imports until


,Group1,Group2,Avg
SF,-9.2,NaN,-9.2
LA,4.0,NaN,4.0
Peoria,-9.9,NaN,-9.9
Lyon,NaN,-8.7,NaN
Paris,NaN,-3.0,NaN
Bruge,NaN,-9.6,NaN
London,NaN,-1.0,NaN
NYC,0.0,0.0,0.0
Boston,-7.0,-7.0,-7.0


In [99]:
a = np.array([np.nan, 3, np.nan])
b = np.array([np.nan, 4, 5])
np.nanmean([a,b], axis=0), np.nansum([a,b], axis=0)

/Users/parrt/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: Mean of empty slice
  This is separate from the ipykernel package so we can avoid doing imports until


(array([nan, 3.5, 5. ]), array([0., 7., 5.]))

## New Impact def

In [207]:
dy = np.array([0.0,12.0,11.0,4.0,2.0])
I = np.array([dy, dy - dy[1], dy - dy[2], dy - dy[3], dy - dy[4]])
I = pd.DataFrame(I, columns=['A','B','C','D','E'], index=['A','B','C','D','E'])
I

,A,B,C,D,E
A,0.0,12.0,11.0,4.0,2.0
B,-12.0,0.0,-1.0,-8.0,-10.0
C,-11.0,1.0,0.0,-7.0,-9.0
D,-4.0,8.0,7.0,0.0,-2.0
E,-2.0,10.0,9.0,2.0,0.0


In [346]:
def all_pairs_delta(avg_per_cat):
    """
    For a vector containing the average Delta per category found
    by merging deltas found across regions, compute the average drop/rise in y
    if we move from all other categories. If we are talking about US state temperatures,
    and leave on an airplane from a random state, what is expected bump or drop
    in temperature when arriving in, say, AZ?  That is the average change in y temp from
    all other states to AZ.
    
    Return array of deltas has same size as arg avg_per_cat.
    """
    avg_pairwise_delta = np.full_like(avg_per_cat, fill_value=np.nan, dtype=float)
    # first find catcodes (same as index into avg_per_cat). Find all non-NaN
    catcodes = np.where(~np.isnan(avg_per_cat))[0]
    ncodes = len(catcodes) # how many codes have usable data?
#     avg_pairwise_delta = []
    for i,k in enumerate(catcodes):
        # Shift cat k to 0 to find relative bump up/down to others;
        rel_to_k = avg_per_cat - avg_per_cat[k]
        # Since not all cats are present, some of these are still NaN;
        # don't count those nor cat k to itself
        avg_delta_away_from_k = np.nansum(rel_to_k) / (ncodes-1)
        avg_delta_to_k = -avg_delta_away_from_k
#         print("to code",k,rel_to_k, avg_delta_to_k)
        avg_pairwise_delta[k] = avg_delta_to_k
    return avg_pairwise_delta

In [314]:
dy = np.array([0.0,12.0,11.0,4.0,2.0])
all_pairs_delta(dy)

to code 0 [ 0. 12. 11.  4.  2.] -7.25
to code 1 [-12.   0.  -1.  -8. -10.] 7.75
to code 2 [-11.   1.   0.  -7.  -9.] 6.5
to code 3 [-4.  8.  7.  0. -2.] -2.25
to code 4 [-2. 10.  9.  2.  0.] -4.75


array([-7.25,  7.75,  6.5 , -2.25, -4.75])

In [315]:
dy = np.array([np.nan,0.0,12.0,11.0,np.nan,4.0,2.0,np.nan])
all_pairs_delta(dy)

to code 1 [nan  0. 12. 11. nan  4.  2. nan] -7.25
to code 2 [ nan -12.   0.  -1.  nan  -8. -10.  nan] 7.75
to code 3 [ nan -11.   1.   0.  nan  -7.  -9.  nan] 6.5
to code 5 [nan -4.  8.  7. nan  0. -2. nan] -2.25
to code 6 [nan -2. 10.  9. nan  2.  0. nan] -4.75


array([  nan, -7.25,  7.75,  6.5 ,   nan, -2.25, -4.75,   nan])

In [316]:
dy = np.array([0.0, -158.7152658238447, 247.7697623395638, 679.7776429908575, 1531.0979231723556, 1875.5949181876208, 1526.352483930149, 1504.4477097847735])
deltas = all_pairs_delta(dy)
print(deltas)
np.mean(np.abs(deltas))

to code 0 [   0.         -158.71526582  247.76976234  679.77764299 1531.09792317
 1875.59491819 1526.35248393 1504.44770978] -1029.4750249402107
to code 1 [ 158.71526582    0.          406.48502816  838.49290881 1689.813189
 2034.31018401 1685.06774975 1663.16297561] -1210.8639001674617
to code 2 [-247.76976234 -406.48502816    0.          432.00788065 1283.32816083
 1627.82515585 1278.58272159 1256.67794745] -746.3095822664235
to code 3 [-679.77764299 -838.49290881 -432.00788065    0.          851.32028018
 1195.8172752   846.57484094  824.67006679] -252.58629009351645
to code 4 [-1531.09792317 -1689.813189   -1283.32816083  -851.32028018
     0.           344.49699502    -4.74543924   -26.65021339] 720.3511729710527
to code 5 [-1875.59491819 -2034.31018401 -1627.82515585 -1195.8172752
  -344.49699502     0.          -349.24243426  -371.1472084 ] 1114.06202441707
to code 6 [-1526.35248393 -1685.06774975 -1278.58272159  -846.57484094
     4.74543924   349.24243426     0.           -21.

809.808699366903

In [317]:
dy - np.mean(dy)

array([ -900.79064682, -1059.50591265,  -653.02088448,  -221.01300383,
         630.30727635,   974.80427136,   625.56183711,   603.65706296])

In [318]:
a = np.array([48.96, 29.23,  0.,   39.1,  19.6 ])
m = np.sum(a)/(len(a)-1)
m = np.mean(a)
m, a-m, a-m-np.min(a-m)

(27.377999999999997,
 array([ 21.582,   1.852, -27.378,  11.722,  -7.778]),
 array([48.96, 29.23,  0.  , 39.1 , 19.6 ]))

In [319]:
all_pairs_delta(a)

to code 0 [  0.   -19.73 -48.96  -9.86 -29.36] 26.9775
to code 1 [ 19.73   0.   -29.23   9.87  -9.63] 2.3149999999999995
to code 2 [48.96 29.23  0.   39.1  19.6 ] -34.2225
to code 3 [  9.86  -9.87 -39.1    0.   -19.5 ] 14.6525
to code 4 [ 29.36   9.63 -19.6   19.5    0.  ] -9.722499999999998


array([ 26.9775,   2.315 , -34.2225,  14.6525,  -9.7225])

In [357]:
a = np.array([0, 30])
all_pairs_delta(a)

array([-30.,  30.])

In [367]:
a = np.array([0]+[1]*20)
a

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [368]:
m = np.mean(a)
m

0.9523809523809523

In [369]:
a-m

array([-0.95238095,  0.04761905,  0.04761905,  0.04761905,  0.04761905,
        0.04761905,  0.04761905,  0.04761905,  0.04761905,  0.04761905,
        0.04761905,  0.04761905,  0.04761905,  0.04761905,  0.04761905,
        0.04761905,  0.04761905,  0.04761905,  0.04761905,  0.04761905,
        0.04761905])

In [359]:
all_pairs_delta(a)

array([-1. ,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1])

In [843]:
a = np.array([0,4,6])
b = np.array([np.nan,0,3])

In [844]:
all_pairs_delta(a), all_pairs_delta(b)

(array([-5.,  1.,  4.]), array([nan, -3.,  3.]))

In [845]:
np.nanmean([all_pairs_delta(a), all_pairs_delta(b)], axis=0)

array([-5. , -1. ,  3.5])

In [846]:
v1 = np.nanmean([a,b + 4], axis=0)
v1

array([0. , 4. , 6.5])

In [847]:
v2 = np.nanmean([a,b - 3 + 6], axis=0)
v2

array([0. , 3.5, 6. ])

In [849]:
v = np.mean([v1,v2], axis=0)
v

array([0.  , 3.75, 6.25])

In [850]:
a - v, b - v

(array([ 0.  ,  0.25, -0.25]), array([  nan, -3.75, -3.25]))

In [852]:
b + 4, b - 3 + 6

(array([nan,  4.,  7.]), array([nan,  3.,  6.]))

In [855]:
np.nanmean([b + 4, b - 3 + 6], axis=0)

/Users/parrt/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


array([nan, 3.5, 6.5])

In [856]:
a = np.array([4,6])
b = np.array([0,3])

In [857]:
b - b[0] + a[0]

array([4, 7])

In [858]:
b - b[1] + a[1]

array([3, 6])

In [860]:
np.mean([b - b[0] + a[0], b - b[1] + a[1]], axis=0)

array([3.5, 6.5])

In [862]:
all = []
for i in range(len(a)):
    all.append(b - b[i] + a[i])
np.mean(all, axis=0)

array([3.5, 6.5])